In [156]:
import pandas as pd
import numpy as np
from database import engine
from ingredient_map import create_ingredient_map
from quant_preprocess import query_data
from quant_preprocess import cols_to_lower
from quant_preprocess import get_cols_list
from quant_preprocess import query_and_preprocess_data
from quant_preprocess import shape_data_long
from quant_preprocess import merge_long
from quant_preprocess import pivot_wide
from quant_preprocess import recode_ingredients
import os


INPUT_PATH = os.path.join("..//data", "ingredient_prices_clean.csv")

df_drinks = query_and_preprocess_data()
df_prices = pd.read_csv(INPUT_PATH, header=None)
liquors = ["strdrink","brandy","gin","tequila","vodka","whiskey","flavored rum","flavored vodka","cognac","bourbon","brandy","scotch","grain alcohol"]

In [2]:
def create_dummies(df):
    
    headers = df.columns[1:]
    d = {'strdrink': df['strdrink'].values.tolist()}
    for e in range(0, len(headers)):
        d[headers[e]] = (df[str(headers[e])] > 0).astype(int).values.tolist()
    return pd.DataFrame.from_dict(d)

In [3]:
def summmary_of_oz(df):
   
    return df.describe().transpose().sort_values('mean',
                                ascending = False).head(10)

In [4]:
def summary_of_usage():
    
    df = query_and_preprocess_data()
    df = create_dummies(df)
    dum_df = create_dummies(df)
    headers = dum_df.columns[1:].values.tolist()
    data = dum_df.describe().transpose().sort_values('mean', 
                                    ascending=False).head(10)
        
    return data
    

In [46]:
def get_amount_table():
    df = query_data()
    ingred_cols = get_cols_list(df, "stringredient")
    measure_cols = get_cols_list(df, "strmeasure")
    df[ingred_cols] = cols_to_lower(df, ingred_cols)
    ingredient_long = shape_data_long(
            df, ingred_cols, "stringredient", "", "ingredient"
        )
    measure_long = shape_data_long(df, measure_cols, "strmeasure", "_clean", "amount")
    combined_long = merge_long(ingredient_long, measure_long)
    ingredient_dict = create_ingredient_map()
    recoded_long = recode_ingredients(combined_long, ingredient_dict)
    combined_wide = pivot_wide(recoded_long)
    return combined_wide


In [120]:
def drop_big_drinks():
    df = get_amount_table()
    df = df.set_index("strdrink")
    df = df.transpose()
    for d in df:
        if (df.sum(axis=0)[d] > 6):
            df = df.drop(d,axis=1)
    return df.columns.values.tolist()
    


In [105]:
def prices_list():
    
    df_prices = pd.read_csv(INPUT_PATH, header=None)
    df_prices = df_prices.sort_values(0, ascending=True)
    df_prices = df_prices.transpose()
    prices = df_prices.loc[1].values.tolist()
    return prices

In [7]:
def combine_prices_ingredients():
    ingredients = get_amount_table()
    ingredients = ingredients.transpose()
    ingredients =  ingredients.drop(labels="strdrink",axis=0)
    prices = prices_list()
    ingredients["prices"] = prices
    return ingredients

In [8]:
def get_ingredient_cost():
    df = get_amount_table()
    drinks = df["strdrink"].values.tolist()
    df = combine_prices_ingredients()
    for i in range(0,627):
        df[i] = df[i]*df["prices"]
    df= df.transpose()
    df = df.drop(labels = "prices", axis = 0)
    df["strdrink"] = drinks
    df = df.transpose()
    
    return df

In [11]:
def drop_all_zero_dummies():
    liquors = ["strdrink","brandy","gin","tequila","vodka","whiskey","flavored rum","flavored vodka","cognac","bourbon","brandy","scotch","grain alcohol"]
    dummies = create_dummies(df_drinks)
    dummies = dummies[liquors].replace(0, np.nan)
    dummies = dummies.dropna(how='all', axis=0)
    dummies = dummies.replace(np.nan, 0)
    
    return dummies

In [12]:
drop_all_zero_dummies()

,strdrink,brandy,gin,tequila,vodka,whiskey,flavored rum,flavored vodka,cognac,bourbon,brandy,scotch,grain alcohol
0,1-900-FUK-MEUP,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
1,110 in the shade,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,151 Florida Bushwacker,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,155 Belmont,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,24k nightmare,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
623,Zizi Coin-coin,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
624,Zoksel,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
625,Zombie,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
626,Zorbatini,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [146]:
def model_data():
    liquors = ["brandy","gin","tequila","vodka","whiskey","flavored rum","flavored vodka","cognac","bourbon","brandy","scotch","grain alcohol"]
    data = get_ingredient_cost()
    amounts = get_amount_table()
    data.columns = data.loc['strdrink']
    drinks = data.columns.values.tolist()
    data = data.drop("strdrink", axis=0)
    df_cost = data.transpose()
    cost = df_cost.sum(axis=1).values.tolist()
    amounts = amounts.sum(axis=1).values.tolist()
    d = {
        "strdrink": drinks,
        "cost": cost,
        "total oz": amounts
    }
    drinks = drop_big_drinks()
    df = pd.DataFrame(d).set_index("strdrink").transpose()
    df = df[drinks].transpose()
    dummies = create_dummies(df_drinks).set_index("strdrink")

    dummies = dummies[liquors]
    dummies = dummies.replace(0, np.nan)
    dummies = dummies.dropna(how='all', axis=0)
    dummies = dummies.replace(np.nan, 0)
    model = dummies.merge(df, how="inner",on="strdrink")
    

    
    
    return model

In [148]:
model = model_data()

C:\Users\galon\AppData\Local\Temp\ipykernel_29168\2399288694.py:10: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  amounts = amounts.sum(axis=1).values.tolist()


In [150]:
corr = model.corr()

In [151]:
corr 


,brandy,gin,tequila,vodka,whiskey,flavored rum,flavored vodka,cognac,bourbon,brandy,scotch,grain alcohol,cost,total oz
brandy,1.000000,-0.104537,-0.135729,-0.241144,-0.073345,-0.053524,-0.076479,-0.082499,-0.014468,1.000000,-0.113992,-0.030765,-0.161080,-0.010551
gin,-0.104537,1.000000,-0.123045,-0.272228,-0.184113,-0.076477,-0.175782,-0.007078,-0.162877,-0.104537,-0.162877,-0.043958,0.311000,0.240493
tequila,-0.135729,-0.123045,1.000000,-0.057141,-0.096139,-0.033963,-0.078062,-0.052348,-0.072332,-0.135729,-0.072332,-0.019521,-0.088221,0.019156
vodka,-0.241144,-0.272228,-0.057141,1.000000,-0.155353,-0.067012,-0.154025,-0.103288,-0.098997,-0.241144,-0.142718,-0.038518,-0.322000,-0.027125
whiskey,-0.073345,-0.184113,-0.096139,-0.155353,1.000000,-0.037912,-0.027198,-0.058435,0.047429,-0.073345,-0.016657,-0.021791,-0.115950,-0.140068
flavored rum,-0.053524,-0.076477,-0.033963,-0.067012,-0.037912,1.000000,-0.030783,-0.020643,-0.028523,-0.053524,-0.028523,-0.007698,-0.136026,-0.131387
flavored vodka,-0.076479,-0.175782,-0.078062,-0.154025,-0.027198,-0.030783,1.000000,-0.047448,-0.065561,-0.076479,-0.065561,-0.017694,-0.157287,-0.034656
cognac,-0.082499,-0.007078,-0.052348,-0.103288,-0.058435,-0.020643,-0.047448,1.000000,-0.043965,-0.082499,-0.043965,-0.011865,0.177023,-0.067546
bourbon,-0.014468,-0.162877,-0.072332,-0.098997,0.047429,-0.028523,-0.065561,-0.043965,1.000000,-0.014468,0.020848,-0.016395,-0.010634,-0.048928
brandy,1.000000,-0.104537,-0.135729,-0.241144,-0.073345,-0.053524,-0.076479,-0.082499,-0.014468,1.000000,-0.113992,-0.030765,-0.161080,-0.010551


In [152]:
df["cost"].corr(df["total oz"])
#justification for including total oz in regression

NameError: name 'df' is not defined

In [159]:
def check_covar_costs():
    df_prices = pd.read_csv(INPUT_PATH, header=None)
    df_prices = df_prices.transpose()
    df_prices.columns = df_prices.loc[0]
    df_prices = df_prices.drop(0)
    df_prices = df_prices[["brandy","gin","tequila","vodka","whiskey","flavored rum","flavored vodka","cognac","bourbon","brandy","scotch","grain alcohol"]]
    return df_prices

In [160]:
check_covar_costs()

,brandy,gin,tequila,vodka,whiskey,flavored rum,flavored vodka,cognac,bourbon,brandy,scotch,grain alcohol
1,0.394,0.788,0.659687,0.391845,0.57,0.5,0.39668,2.263396,1.1,0.394,2.3,0.246667
